<a href="https://colab.research.google.com/github/Tixonmavrin/Zindi-Zimnat-Insurance-Recommendation-Challenge/blob/master/Baseline1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Zimnat Insurance Recommendation Challenge
Can you predict which insurance products existing clients will want next?
###Baseline with catboost

### Get data

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
%matplotlib inline

import sys
import os
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, VotingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np
import pandas as pd
# from time import time
import time
import pprint
import joblib
from sklearn.model_selection import KFold
from xgboost.sklearn import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
import warnings
import logging
warnings.filterwarnings("ignore")

from scipy.optimize import minimize
# Classifiers
from catboost import CatBoostClassifier

from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.optimize import minimize
from sklearn.metrics import log_loss

# Model selection
from sklearn.model_selection import StratifiedKFold

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l1_l2

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

# New Section

### New format

In [3]:
X_train = []
X_train_columns = train.columns
c = 0
for v in train.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
        c+=1
        for k in range(len(binary)):
            if k == i:
                binary_transformed = list(copy.copy(binary))
                binary_transformed[i] = 0
                X_train.append(list(info) + binary_transformed + [X_train_columns[8+k]] + [c])

X_train = pd.DataFrame(X_train)
X_train.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'ID2']

In [4]:
X_test = []
true_values = []
c = 0
for v in test.values:
    c += 1
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    X_test.append(list(info) + list(binary) + [c])
    for k in test.columns[8:][index]:
        true_values.append(v[0] + ' X ' + k)

X_test = pd.DataFrame(X_test)
X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'ID2']

### Transform data

In [5]:
features_train = []
features_test = []
columns = []

append_features = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 
'N2MW', 'AHXO','BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 
'ECY3', 'ID', 'ID2', 'join_date', 'sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',
'birth_year']
for v in append_features:
    features_train.append(X_train[v].values.reshape(-1, 1))
    features_test.append(X_test[v].values.reshape(-1, 1))
    columns.append(np.array([v]))

y_train = X_train[['product_pred']]

In [6]:
features_train = np.concatenate(features_train, axis=1)
features_test = np.concatenate(features_test, axis=1)
columns = np.concatenate(np.array(columns))

X_train = pd.DataFrame(features_train)
X_train.columns = columns
X_test = pd.DataFrame(features_test)
X_test.columns = columns

### New features

In [7]:
X_train['date1'] = X_train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_train['date2'] = X_train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_train['date3'] = X_train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_train.drop('join_date', axis=1, inplace=True)

X_test['date1'] = X_test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_test['date2'] = X_test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_test['date3'] = X_test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_test.drop('join_date', axis=1, inplace=True)

X_train['date_diff'] = X_train['date3'] - X_train['birth_year']
X_test['date_diff'] = X_test['date3'] - X_test['birth_year']

### Change types

In [8]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data = X_train.append(X_test)
for v in ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',]:
    data.loc[:,v] = le.fit_transform(data.loc[:,v])
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [10]:
le.fit(y_train.iloc[:,0])
y_train = pd.DataFrame(le.transform(y_train.iloc[:,0]))
y_train.columns = ['target']

In [11]:
pd.set_option('display.max_columns', None)
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [12]:
pd.set_option('display.max_columns', None)
X_train.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,date1,date2,date3,date_diff
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4WKQSBB,1,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4WKQSBB,2,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,CP5S02H,3,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,CP5S02H,4,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2YKDILJ,5,1,6,3,185,1,1991,1.0,6.0,2013.0,22.0


In [13]:
data = X_train.append(X_test)

In [14]:
freq_1 = data['marital_status'].value_counts()
data['marital_status_fe'] = data['marital_status'].map(freq_1)

freq_2 = data['branch_code'].value_counts()
data['branch_code_fe'] = data['branch_code'].map(freq_2)

freq_3 = data['occupation_category_code'].value_counts()
data['occupation_category_code_fe'] = data['occupation_category_code'].map(freq_3)
  

In [15]:
## Squaring the features 

data['marital_status_sq'] = data['marital_status']**2
data['branch_code_sq'] = data['branch_code']**2
data['occupation_code_sq'] = data['occupation_code']**2
data['occupation_category_code_sq'] = data['occupation_category_code']**2
data['date_diff_sq'] = data['date_diff']**2

data['marital_status_cube'] = data['marital_status']**3
data['branch_code_cube'] = data['branch_code']**3
data['occupation_code_cube'] = data['occupation_code']**3
data['occupation_category_code_cube'] = data['occupation_category_code']**3
data['date_diff_cube'] = data['date_diff']**3

data['marital_status_log'] = (data['marital_status']+1).transform(np.log)
data['branch_code_log'] = (data['branch_code']+1).transform(np.log)
data['occupation_code_cube'] = (data['occupation_code']+1).transform(np.log)
data['occupation_category_code_log'] = (data['occupation_category_code']+1).transform(np.log)
data['date_diff_log'] = (data['date_diff']+1).transform(np.log)

data['sex*marital_status'] = data['sex'] * data['marital_status']
data['occupation_code*occupation_cat_code'] = data['occupation_code'] * data['occupation_category_code']
# data = data.drop(['marital_status', 'branch_code', 'occupation_category_code'], axis=1)
# X_train = data[:X_train.shape[0]]
# X_test = data[-X_test.shape[0]:]

In [16]:
data.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,date1,date2,date3,date_diff,marital_status_fe,branch_code_fe,occupation_category_code_fe,marital_status_sq,branch_code_sq,occupation_code_sq,occupation_category_code_sq,date_diff_sq,marital_status_cube,branch_code_cube,occupation_code_cube,occupation_category_code_cube,date_diff_cube,marital_status_log,branch_code_log,occupation_category_code_log,date_diff_log,sex*marital_status,occupation_code*occupation_cat_code
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4WKQSBB,1,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0,59731,3130,48918,4,0,361,25,1024.0,8,0,2.995732,125,32768.0,1.098612,0.000000,1.791759,3.496508,0,95
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4WKQSBB,2,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0,59731,3130,48918,4,0,361,25,1024.0,8,0,2.995732,125,32768.0,1.098612,0.000000,1.791759,3.496508,0,95
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,CP5S02H,3,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0,59731,8972,48918,4,121,361,25,1444.0,8,1331,2.995732,125,54872.0,1.098612,2.484907,1.791759,3.663562,0,95
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,CP5S02H,4,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0,59731,8972,48918,4,121,361,25,1444.0,8,1331,2.995732,125,54872.0,1.098612,2.484907,1.791759,3.663562,0,95
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2YKDILJ,5,1,6,3,185,1,1991,1.0,6.0,2013.0,22.0,6905,21481,14842,36,9,34225,1,484.0,216,27,5.225747,1,10648.0,1.945910,1.386294,0.693147,3.135494,6,185


In [17]:
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [18]:
train = pd.concat([X_train, y_train], axis=1)
train

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,date1,date2,date3,date_diff,marital_status_fe,branch_code_fe,occupation_category_code_fe,marital_status_sq,branch_code_sq,occupation_code_sq,occupation_category_code_sq,date_diff_sq,marital_status_cube,branch_code_cube,occupation_code_cube,occupation_category_code_cube,date_diff_cube,marital_status_log,branch_code_log,occupation_category_code_log,date_diff_log,sex*marital_status,occupation_code*occupation_cat_code,target
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4WKQSBB,1,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0,59731,3130,48918,4,0,361,25,1024.0,8,0,2.995732,125,32768.0,1.098612,0.000000,1.791759,3.496508,0,95,19
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4WKQSBB,2,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0,59731,3130,48918,4,0,361,25,1024.0,8,0,2.995732,125,32768.0,1.098612,0.000000,1.791759,3.496508,0,95,12
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,CP5S02H,3,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0,59731,8972,48918,4,121,361,25,1444.0,8,1331,2.995732,125,54872.0,1.098612,2.484907,1.791759,3.663562,0,95,19
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,CP5S02H,4,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0,59731,8972,48918,4,121,361,25,1444.0,8,1331,2.995732,125,54872.0,1.098612,2.484907,1.791759,3.663562,0,95,12
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2YKDILJ,5,1,6,3,185,1,1991,1.0,6.0,2013.0,22.0,6905,21481,14842,36,9,34225,1,484.0,216,27,5.225747,1,10648.0,1.945910,1.386294,0.693147,3.135494,6,185,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66348,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,MVXG2LC,66349,1,2,6,46,5,1985,1.0,2.0,2019.0,34.0,59731,975,48918,4,36,2116,25,1156.0,8,216,3.850148,125,39304.0,1.098612,1.945910,1.791759,3.555348,2,230,12
66349,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,BUQ8DQN,66350,1,2,11,19,5,1989,1.0,7.0,2019.0,30.0,59731,8972,48918,4,121,361,25,900.0,8,1331,2.995732,125,27000.0,1.098612,2.484907,1.791759,3.433987,2,95,19
66350,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,BUQ8DQN,66351,1,2,11,19,5,1989,1.0,7.0,2019.0,30.0,59731,8972,48918,4,121,361,25,900.0,8,1331,2.995732,125,27000.0,1.098612,2.484907,1.791759,3.433987,2,95,12
66351,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,DEJBWYU,66352,1,2,4,30,1,1975,1.0,3.0,2020.0,45.0,59731,9428,14842,4,16,900,1,2025.0,8,64,3.433987,1,91125.0,1.098612,1.609438,0.693147,3.828641,2,30,19


In [19]:
train = train.drop(['ID', 'ID2'], axis=1)
X_test = X_test.drop(['ID', 'ID2'], axis=1)

In [20]:
train.to_csv("processed_train.csv")
X_test.to_csv("processed_test.csv")

In [21]:
def elapsed_time(start_time, end_time):
    elapsed_sec = end_time - start_time
    h = int(elapsed_sec / (60 * 60))
    m = int((elapsed_sec % (60 * 60)) / 60)
    s = int(elapsed_sec % 60)
    return "{}:{:>02}:{:>02}".format(h, m, s)


def standarize_feature(train_df, test_df, cols):
    scaler = StandardScaler()
    for col in cols:
        train_df[col] = scaler.fit_transform(train_df[col].reshape(-1,1).astype(np.float32))
        test_df[col] = scaler.transform(test_df[col].reshape(-1,1).astype(np.float32))
    return None


def extend_bounds(bins):
    bins[0] = bins[0] - 1
    bins[-1] = bins[-1] + 1


def discretize_feature(train_df, test_df, cols, num_bins, how='equal_freq'):
    if how == 'equal_width':
        for col in cols:
            group_names = range(num_bins)
            train_df[col], bins = pd.cut(train_df[col], num_bins, labels=group_names, retbins=True)
            test_df[col] = pd.cut(test_df[col], bins, labels=group_names)
    elif how == 'equal_freq':
        for col in cols:
            freq_per_bin = train_df[col].shape[0] / num_bins
            values = sorted(train_df[col])
            bins = [values[0]]
            for i in range(1, num_bins + 1):
                if i < num_bins:
                    bins.append(values[i * freq_per_bin - 1])
                else:
                    bins.append(values[-1])
            group_names = range(num_bins)

            i = 1
            while i < len(bins):
                if bins[i] == bins[i-1]:
                    del bins[i]
                    del group_names[i]
                else:
                    i += 1
            extend_bounds(bins)
            train_df[col] = pd.cut(train_df[col], bins, labels=group_names)
            test_df[col] = pd.cut(test_df[col], bins, labels=group_names)
    return None


def load_data(train_data_path='data/train.csv', test_data_path = 'data/test.csv'):
    train_df = pd.read_csv(train_data_path, sep=',', index_col=0, header=0)
    test_df = pd.read_csv(test_data_path, sep=',', index_col=0, header=0)
    
#     train_df['target'] = train_df['target'].str[-1].astype(int) - 1
        
    return train_df, test_df


In [22]:
classes = np.array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

def process_data(train_df, test_df, ylabel='target', standarization=False, discretization=False, transform=None):
    numerical_features = train_df.columns

    if standarization:
        standarized_features = numerical_features
        standarize_feature(train_df, test_df, standarized_features)
        
    if discretization:
        discretized_features = numerical_features
        discretize_feature(train_df, test_df, discretized_features, num_bins=10, how='equal_freq')
    
    X = train_df.drop(ylabel, axis=1)
    y = train_df[ylabel]
    X_submission = test_df
    
    if transform == 'log':
        X = np.log1p(X)
        X_submission = np.log1p(X_submission)
    elif transform == 'sqrt':
        X = np.sqrt(X + 3.0 / 8)
        X_submission = np.sqrt(X_submission + 3.0 / 8)
    elif transform == 'pca':
        pca = PCA(n_components=3).fit(X)
        X = pca.transform(X)
        X_submission = pca.transform(X_submission)
    elif transform == 'tsne':
        tsne = TSNE(n_components=3).fit(X)
        X = tsne.transform(X)
        X_submission = tsne.transform(X_submission)
    elif transform == 'pca+':
        pca = PCA(n_components=3).fit(X)
        X = np.hstack((X, pca.transform(X)))
        X_submission = np.hstack((X, pca.transform(X)))
    elif transform == 'tsne+':
        tsne = TSNE(n_components=3).fit(X)
        X = np.hstack((X, tsne.transform(X)))
        X_submission = np.hstack((X_submission, tsne.transform(X_submission)))        
    return X, y, X_submission


def evaluate(y, y_pred):
    logloss = log_loss(y, y_pred, labels=classes)
    return logloss


def models_CV_train(models, X, y, X_submission, n_classes, n_folds=5):
    summary = {}

    skf = list(StratifiedKFold(n_folds, random_state=0).split(X, y))
    
    stack_train = np.zeros((X.shape[0], n_classes, len(models)))
    stack_test = np.zeros((X_submission.shape[0], n_classes, len(models)))
    
    for i, model in enumerate(models):
        print("Model %d:" % i, model)
        
        avg_logloss = 0
        
        stack_test_model_i = np.zeros((X_submission.shape[0], n_classes, len(skf)))
        for j, (train_idx, test_idx) in enumerate(skf):
            print("  Fold %d" % j)
            X_train = X.iloc[train_idx]
            y_train = y.iloc[train_idx]
            X_test = X.iloc[test_idx]
            y_test = y.iloc[test_idx]

            model.fit(X_train, y_train)
            
            y_test_pred = model.predict_proba(X_test)          
            stack_train[test_idx, :, i] = y_test_pred
            
            logloss = evaluate(y_test, y_test_pred)
            avg_logloss += logloss
            print("logloss: %f" % logloss)
            
            y_submission_pred = model.predict_proba(X_submission)           
            stack_test_model_i[:, :, j] = y_submission_pred
        
        avg_logloss = avg_logloss / n_folds
        print ("model average logloss: %f" % avg_logloss)
        summary[i] = avg_logloss
        
        stack_test[:, :, i] = stack_test_model_i.mean(axis=2)

    return np.swapaxes(stack_train, 1, 2).reshape((X.shape[0], -1)), np.swapaxes(stack_test, 1, 2).reshape((X_submission.shape[0], -1)), summary


In [23]:
def stacking_models_CV_train(models, X, y, X_submission, n_classes, n_folds=5):
    skf = list(StratifiedKFold(n_folds, random_state=42).split(X, y))
    
    y_submission_pred = np.zeros((X_submission.shape[0], n_classes, len(models)))
        
    for i, model in enumerate(models):
        print ("Stacking Model %d" % i, model)
        avg_logloss = 0
        y_submission_pred_model_i = np.zeros((X_submission.shape[0], n_classes, len(skf)))
        for j, (train_idx, test_idx) in enumerate(skf):
            print ("  Fold", j); sys.stdout.flush()
            X_train = X[train_idx]
            y_train = y[train_idx]
            X_test = X[test_idx]
            y_test = y[test_idx]

            model.fit(X_train, y_train)

            y_test_pred = model.predict_proba(X_test)          

            logloss = evaluate(y_test, y_test_pred)
            avg_logloss += logloss
            print ("  logloss: %f" % logloss)

            y_submission_pred_model_i[:, :, j] = model.predict_proba(X_submission)

        avg_logloss = avg_logloss / n_folds
        print ("model average logloss: %f" % avg_logloss)

        y_submission_pred[:, :, i] = y_submission_pred_model_i.mean(axis=2)

    return y_submission_pred  


def create_2_layer_keras_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dropout(0.05, input_shape=(input_dim,)))
    model.add(Dense(512, kernel_initializer='glorot_normal', activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    model.add(Dropout(0.5))
    model.add(Dense(256, kernel_initializer='glorot_normal', activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    model.add(Dropout(0.5))
    model.add(Dense(output_dim, kernel_initializer='glorot_normal', activation='softmax', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', 'categorical_crossentropy'])
    return model


def create_3_layer_keras_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dropout(0.05, input_shape=(input_dim,)))
    model.add(Dense(1024, kernel_initializer='glorot_normal', activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    model.add(Dropout(0.5))    
    model.add(Dense(512, kernel_initializer='glorot_normal', activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    model.add(Dropout(0.5))
    model.add(Dense(256, kernel_initializer='glorot_normal', activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    model.add(Dropout(0.5))
    model.add(Dense(output_dim, kernel_initializer='glorot_normal', activation='softmax', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'categorical_crossentropy'])
    return model

In [24]:
start_time = time.time()

logging.basicConfig(level=logging.DEBUG,
                    format='[%(asctime)s]: %(message)s ',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    stream=sys.stdout,
                    filemode="w"
                    )

# load data
logging.info('Load data')
train_df, test_df = load_data(train_data_path='processed_train.csv', test_data_path='processed_test.csv')
X, y, X_submission = process_data(train_df, test_df, transform='sqrt')

# training phase 1
logging.info('Training phase 1')
models = []

# KNN
# models += [KNeighborsClassifier(n_neighbors=2, n_jobs=-1),
#             KNeighborsClassifier(n_neighbors=4, n_jobs=-1),
#             KNeighborsClassifier(n_neighbors=8, n_jobs=-1),
#             KNeighborsClassifier(n_neighbors=16, n_jobs=-1),
#             KNeighborsClassifier(n_neighbors=32, n_jobs=-1),
#             KNeighborsClassifier(n_neighbors=64, n_jobs=-1),
#             KNeighborsClassifier(n_neighbors=128, n_jobs=-1),
#             KNeighborsClassifier(n_neighbors=256, n_jobs=-1),
#             KNeighborsClassifier(n_neighbors=512, n_jobs=-1),
#             KNeighborsClassifier(n_neighbors=1024, n_jobs=-1)]

# Logistic Regression
# models += [LogisticRegression(penalty='l2', solver='lbfgs', n_jobs=-1, C=0.01, class_weight=None, multi_class='multinomial')]

# Random Forest
# models += [RandomForestClassifier(criterion='gini', n_jobs=-1, n_estimators=250, max_features=18, min_samples_split=5, max_depth=46, class_weight=None)]

# CatBoost Classifier
models += [CatBoostClassifier()]

# sklearn extra-trees classifier
# models += [ExtraTreesClassifier(criterion='gini', n_estimators=500, n_jobs=-1)]

# xgboost
models += [XGBClassifier(learning_rate=0.1675, max_depth=10, gamma=1.976, n_estimators=550),
           XGBClassifier(learning_rate=0.1675, max_depth=10, gamma=1.976, n_estimators=550)]

# NN
# models += [KerasClassifier(build_fn=create_2_layer_keras_model, input_dim=50, output_dim=21, nb_epoch=100, batch_size=256, verbose=0),
#             KerasClassifier(build_fn=create_3_layer_keras_model, input_dim=50, output_dim=21, nb_epoch=100, batch_size=256, verbose=0)]

# models += [KerasClassifier(build_fn=create_2_layer_keras_model, input_dim=X.shape[1], output_dim=21, nb_epoch=100, batch_size=256, verbose=0)]

logging.info('List of models to train and ensemble:')
for model in models:
    print (model)
num_models = len(models)

train_models_pred, test_models_pred, summary = models_CV_train(models, X, y, X_submission, n_classes=21, n_folds=5)
# save phase 1 intermediate prediction results
# np.savetxt('train_models_pred.csv', train_models_pred)
# np.savetxt('test_models_pred.csv', test_models_pred)

logging.info('Summary for phase 1 model performance:')
for i in sorted(summary.keys()):
    print ('Model %d: %f' % (i, summary[i]))

# training phase 2
logging.info('Training phase 2: stacking model predictions from phase 1')
stacking_models = []

# stacking models' predictions using xgboost
stacking_models +=[XGBClassifier(learning_rate=0.1675, max_depth=10, gamma=1.976, n_estimators=550)]

# stacking models' predictions using NN
# stacking_models += [KerasClassifier(build_fn=create_2_layer_keras_model, input_dim=num_models * 21, output_dim=21, nb_epoch=2, batch_size=256, verbose=0)]


[2020-08-26 03:07:58]: Load data 
[2020-08-26 03:07:58]: Training phase 1 
[2020-08-26 03:07:58]: List of models to train and ensemble: 
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=1.976,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.1675, max_delta_step=None, max_depth=10,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=550, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=1.976,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
       

In [1]:
y_submission_pred = stacking_models_CV_train(stacking_models, train_models_pred, y, test_models_pred, n_classes=21, n_folds=5)
y_submission_pred = y_submission_pred.mean(axis=2)

y_test = pd.DataFrame(y_submission_pred)
y_test.columns = le.inverse_transform(y_test.columns)

NameError: name 'stacking_models_CV_train' is not defined

In [2]:
X_test = data[-X_test.shape[0]:]
answer_mass = []
for i in range(X_test.shape[0]):
    id = X_test['ID'].iloc[i]
    for c in y_test.columns:
        answer_mass.append([id + ' X ' + c, y_test[c].iloc[i]])


NameError: name 'data' is not defined

In [3]:
df_answer = pd.DataFrame(answer_mass)
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
    if df_answer['ID X PCODE'].iloc[i] in true_values:
        df_answer['Label'].iloc[i] = 1.0
        
df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission70.csv', index=False)

# write submission
#     logging.info("Write submission: averaging stacking models' predictions")
#     timestamp = time.strftime("%Y-%m-%d_%H:%M:%S")
#     columns = ['Class_' + str(i + 1) for i in range(21)]
#     submission_df = pd.DataFrame(y_submission_pred, columns=columns)
#     submission_df.index = submission_df.index + 1
#     submission_df.to_csv('submission_%s.csv' % timestamp, sep=',', index_label='id')

end_time = time.time()
logging.info("Run complete: %s elapsed" % elapsed_time(start_time, end_time))

NameError: name 'pd' is not defined

Submit